# Time-Resolved Carbon Factors

This notebook combines Energy Hub modelling in BESOS with analysis of the wider power system using SILVER [[1]].  
SILVER solves the optimal power flow problem for an entire electricty grid.  
From this we can extract the carbon emission every hour, which depends on the generation mix at that moment.   
This is obtained by running `uc_results_analysis_Storage_Remuneration_w_mustrun.py` to obtain `hourly_GHG_emission.csv`.  
After some processing, this gives the carbon factor time series that is used in place of the annual-averaged carbon factors typically used in the Energy Hub model.

[1]:https://doi.org/10.1016/j.energy.2017.07.027

In [1]:
import os
import csv
from pyehub.outputter import pretty_print
from pyehub.energy_hub.ehub_model import EHubModel
from pyehub.energy_hub.utils import constraint, constraint_list
from pprint import pprint as pp
from besos import pyehub_funcs as pf
import pandas as pd

## Importing CSV
First the .csv file is read in and the date column is given a column name.  
The type of each generator is removed from the table as the type is not important for PyEHub.

In [2]:
raw_data = pd.read_csv('hourly_GHG_emission.csv') 
raw_data.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
raw_data = raw_data.set_index('Date')
raw_data=raw_data.drop(['kind'],axis=0)
raw_data.head()

,Gas_CC_1,Gas_CC_2,Gas_CC_3,Gas_CC_4,Gas_CC_5,Gas_CC_6,Gas_CC_7,Gas_CC_8,Gas_Simple_1,Gas_Simple_2,...,Solar_5,Solar_6,Solar_7,Solar_8,Solar_9,Solar_10,Niagara_PHS,Dryden_d,Toronto_PHS,Toronto_Hydrogen
Date,,,,,,,,,,,,,,,,,,,,,
2012-05-27 00:00:00,490000.0,490000.0,490000.0,245000.0,245000.0,245000.0,245000.0,303310.0000000008,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0
2012-05-27 01:00:00,490000.0,490000.0,245000.0,122500.0,122500.0,122500.0,122500.0,151900.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-05-27 02:00:00,490000.0,490000.0,490000.0,245000.0,245000.0,245000.0,245000.0,303310.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-05-27 03:00:00,490000.0,490000.0,490000.0,245000.0,245000.0,245000.0,245000.0,303310.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2012-05-27 04:00:00,490000.0,490000.0,490000.0,245000.0,245000.0,245000.0,245000.0,303310.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## CSV Reformating
To ensure no type errors all the columns are converted into floats.  
The index is switched to the time steps used by PyEHub and the specific dates are removed.  
*This can be changed depending on how the particular model handles the timesteps*.  
A unit conversion ensures the carbon factors are in the correct units for the PyEHub model.  
The output is then converted into a dictionary where the key is each timestep and the value is the hourly carbon factor.

In [3]:
data=raw_data.apply(pd.to_numeric)

data=data.reset_index()
data=data.drop(['Date'],axis=1)
#data=data[:24] # first day only
data=data.mean(axis=1)/100000
dict_w_index=data.to_dict()

carbon_factor_input = dict_w_index

## Creating a PyEHub model
The PyEHub model is created normally.  
To show the difference in results the unmodified example model is solved.  

In [4]:
hub = EHubModel(excel='test_file_CO2.xlsx')
oldhub = hub.solve()

## Modifying Carbon Factors and Time Series
The carbon factors are only applied to import streams and carbon credits are only applied to export streams.  
The carbon factor for the particular stream is tied to a time series by overwritting the value of the carbon factor with the key for the time series.  
The formated values from the csv are then inputted into the model as a time series.

In [5]:
print("Import Streams:")
print(hub.import_streams)
print()

print("Carbon Factors Before:")
print(hub.CARBON_FACTORS)
print()
hub.CARBON_FACTORS['Grid']='Grid_co2_factor'
print("Carbon Factors After:")
print(hub.CARBON_FACTORS)
print()

print("Time Series Before:")
print(hub.TIME_SERIES)
print()
hub.TIME_SERIES['Grid_co2_factor']=carbon_factor_input
print("Time Series After:")
print(hub.TIME_SERIES)

Import Streams:
['Grid', 'Gas']

Carbon Factors Before:
{'Elec': 0.0, 'Heat': 0.0, 'Irradiation': 0.0, 'Grid': 0.35, 'Gas': 0.194, 'PV_Elec': 0.0}

Carbon Factors After:
{'Elec': 0.0, 'Heat': 0.0, 'Irradiation': 0.0, 'Grid': 'Grid_co2_factor', 'Gas': 0.194, 'PV_Elec': 0.0}

Time Series Before:
{'Elec': {0: 1.0, 1: 4.0, 2: 4.0, 3: 4.0, 4: 4.0, 5: 4.0, 6: 4.0, 7: 4.0, 8: 4.0, 9: 4.0, 10: 4.0}, 'Heat': {0: 20.0, 1: 20.0, 2: 20.0, 3: 20.0, 4: 20.0, 5: 20.0, 6: 20.0, 7: 12.0, 8: 12.0, 9: 12.0, 10: 12.0}, 'Irradiation': {0: 0.0, 1: 0.019, 2: 0.052, 3: 0.092, 4: 0.115, 5: 0.116, 6: 0.095, 7: 0.057, 8: 0.006, 9: 0.0, 10: 0.0}}

Time Series After:
{'Elec': {0: 1.0, 1: 4.0, 2: 4.0, 3: 4.0, 4: 4.0, 5: 4.0, 6: 4.0, 7: 4.0, 8: 4.0, 9: 4.0, 10: 4.0}, 'Heat': {0: 20.0, 1: 20.0, 2: 20.0, 3: 20.0, 4: 20.0, 5: 20.0, 6: 20.0, 7: 12.0, 8: 12.0, 9: 12.0, 10: 12.0}, 'Irradiation': {0: 0.0, 1: 0.019, 2: 0.052, 3: 0.092, 4: 0.115, 5: 0.116, 6: 0.095, 7: 0.057, 8: 0.006, 9: 0.0, 10: 0.0}, 'Grid_co2_factor': {0

## Results
The model is then recompiled with the new time resolved carbon factors.  
We can see that the outputs have changed.

In [6]:
hub.recompile()
newhub=hub.solve()
print(oldhub['solution']['total_carbon'])
print(newhub['solution']['total_carbon'])

33.7551
35.6157


In [7]:
pp(newhub['solution'])

{'ANNUAL_MAINTENANCE_STORAGE_COSTS': {'Battery': 0, 'Hot Water Tank': 0},
 'BIG_M': 99999,
 'Battery': 0.0,
 'Boiler': 10.0,
 'CARBON_CREDITS': {'Elec': 0.0,
                    'Gas': '',
                    'Grid': '',
                    'Heat': '',
                    'Irradiation': '',
                    'PV_Elec': 0.3},
 'CARBON_FACTORS': {'Elec': 0.0,
                    'Gas': 0.194,
                    'Grid': 'Grid_co2_factor',
                    'Heat': 0.0,
                    'Irradiation': 0.0,
                    'PV_Elec': 0.0},
 'CHARGING_EFFICIENCY': {'Battery': 0.99, 'Hot Water Tank': 0.99},
 'CHP': 5.0,
 'CONVERSION_EFFICIENCY': {'Boiler': {'Elec': 0.0,
                                      'Gas': -1.0,
                                      'Grid': 0.0,
                                      'Heat': 0.94,
                                      'Irradiation': 0.0,
                                      'PV_Elec': 0.0},
                           'CHP': {'Elec': 0.3,
 